# Camada SILVER

## Informações Gerais
  | Information | Details |
  |---------|------|
  |Table Name | silver.pedidos|
  |Origem | bronze.pedidos / bronze.estabalecimentos|

## Histórico de Atualizações
|Data | Desenvolvido por | Motivo |
|:------:|--------|-------|
|mm/dd/aaaa|Data Eng.|details|



In [0]:
### Import of libs

from pyspark.sql.functions import current_date, current_timestamp, expr

In [0]:
# Funções

## Função que aplica comentário na tabela!

def adicionaComentariosTabela(a,b,c,d):
    spark.sql(f"COMMENT ON TABLE {a}.{b} IS 'C'")
    for key,value in d.items():
        sqlaux = f"ALTER TABLE {a}.{b} CHANGE COLUMN {key} COMMENT '{value}'"
        spark.sql(sqlaux)

In [0]:
## Silver Table creation estabelecimento

### Name all var

database = "silver"
tabela = "pedidos"

### Table Creation

df_pedidos = spark.sql(
f"""
WITH pedidos AS (
SELECT
    PedidoID AS id_pedido,
    EstabelecimentoID AS id_estabelecimento,
    Produto,
    quantidade_vendida,
    Preco_Unitario,
    data_venda
FROM 
    bronze.pedidos
),
estabelecimentos AS (
SELECT 
    Local AS nome_estabalecimento,
    Email,
    EstabelecimentoID AS id_estabelecimento,
    Telefone
FROM
    bronze.estabelecimentos
)
SELECT
    CAST(ped.id_pedido AS int),
    CAST(ped.id_estabelecimento AS int),
    ped.Produto,
    CAST(ped.quantidade_vendida AS int),
    CAST(ped.Preco_Unitario AS DECIMAL(20,2)),
    CAST(ped.data_venda AS DATE),
    est.nome_estabalecimento,
    est.Email,
    est.Telefone
FROM
    pedidos ped
LEFT JOIN 
    estabelecimentos est
ON ped.id_estabelecimento = est.id_estabelecimento


"""
)



In [0]:
## Na camada SILVER é necessário a existência de Metadados, para maior compreensão das colunas. 
#Isso pode ser feito de forma MANUAL ou automática

# para automatico, ver a função AdicionaComentariosTabela

comentario_tabela = "esta tabela é uma tabela corporativa para XXX usos"

lista_comentario_colunas ={
'id_pedido' : 'ID do Pedido (PK)',
'id_estabelecimento' : 'ID do Establ (FK)',
'Produto' : 'Nome do Produto',
'quantidade_vendida': 'Quantidade vendida na operação',
'Preco_Unitario': 'Preço unitario lançado',
'data_venda': 'data',
'nome_estabalecimento': 'nome fantasia',
'Email': 'string',
'Telefone': 'com DDD'
}

print('Dados Gravados!')

Dados Gravados!


In [0]:
### Save the dateframe as a table

df_pedidos.write \
    .format('delta') \
    .mode('overwrite') \
    .clusterBy("id_pedido") \
    .option('mergeSchema','true') \
    .option('overwriteSchema','true') \
    .saveAsTable(f'{database}.{tabela}')
adicionaComentariosTabela(database,tabela,comentario_tabela,lista_comentario_colunas)
print("Dados gravados com sucesso")

Dados gravados com sucesso


In [0]:
spark.sql(f"OPTIMIZE {database}.{tabela}")
print("Otimização Finalizada")

Otimização Finalizada
